# Goal 

Implement a sequence assembly algorithm to assemble fragmented reads

## Resources 

https://stackoverflow.com/questions/58598805/how-to-find-the-longest-common-suffix-prefix-between-two-strings-in-python-in-a

In [1]:
import numpy as np
import pandas as pd
import csv
import itertools 

Read in the table and create dataframe 

In [2]:
df = pd.read_csv('fragmented_reads_table.csv')
seq_list = df['Sequence'].to_list()

# overlap 

This function returns the longest suffix of a that matches b. 

For example, if you had string XYZABC and ABCDEF the overlap would be 3. 

In [3]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

# find start & end 

need to find the start & end of the sequence to know where to begin/end assembly 

In [4]:
def find_start(alphabet, match_length): 

    start_list = alphabet.copy() 
    for pair in itertools.permutations(alphabet,2): 
            #print(pair)
            for i in range(len(pair)-1): 
                overlap_len = (overlap(pair[i],pair[i+1],match_length))
                if (overlap_len > 0): 
                    #print(pair[i+1])
                    pair_to_remove = pair[i+1]
                    if(pair_to_remove in start_list):    
                        #print("here")
                        start_list.remove(pair_to_remove)

    return(start_list)

In [26]:
find_start(seq_list,5)

['CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGT']

In [6]:
seq_list[0]

'CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGT'

In [7]:
print(seq_list[0]== find_start(seq_list,5)[0])

True


Says this is the start 

CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGT

In [8]:
def find_end(alphabet, match_length): 

    start_list = alphabet.copy() 
    for pair in itertools.permutations(alphabet,2): 
            #print(pair)
            for i in range(len(pair)-1): 
                overlap_len = (overlap(pair[i],pair[i+1],match_length))
                if (overlap_len > 0): 
                    #print(pair[i+1])
                    pair_to_remove = pair[i]
                    if(pair_to_remove in start_list):    
                        #print("here")
                        start_list.remove(pair_to_remove)

    return(start_list)

In [9]:
find_end(seq_list,5)

['CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCTACGGGCAACGGGTTATACTTAGCTGCAACCAACGCCTTTCCACATGTTTGAGAA']

Says this is the end 

CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCTACGGGCAACGGGTTATACTTAGCTGCAACCAACGCCTTTCCACATGTTTGAGAA

In [10]:
print(seq_list[-1] == find_end(seq_list,5)[0])

True


# Layout 

Now we can create the dataframe where we work. The dataframe will contain two sequences and their overlap value. 

In [11]:
pair_dict = [] 

for pair in itertools.permutations(seq_list,2):
    my_pair = (pair[0], pair[1] ,overlap(pair[0], pair[1],50))
    pair_dict.append(my_pair)

In [12]:
from pandas import DataFrame
df = DataFrame(pair_dict,columns=['start', 'end', 'overlap'])

Here is that aforementioned dataframe.

In [13]:
df

,start,end,overlap
0,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTAT...,85
1,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,TCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGT...,70
2,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,AGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAAT...,55
3,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,TCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACG...,0
4,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,TGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGA...,0
...,...,...,...
15997,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,CGTCCATGCTCATGTATTTATATGACGGCCAAAAATGGAGATATTA...,0
15998,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,ATTTATATGACGGCCAAAAATGGAGATATTATAGTCGACCAAGTAT...,0
15999,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,AAAAATGGAGATATTATAGTCGACCAAGTATTGGCGTCGAACAACC...,0
16000,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,ATAGTCGACCAAGTATTGGCGTCGAACAACCGCGCCCTGCAGAATC...,0


Now we want to remove all rows where there is no overlap 

In [15]:
updated_dict = ([i for i in pair_dict if i[2] != 0 ])

In [17]:
from pandas import DataFrame
updated_df = DataFrame(updated_dict,columns=['start', 'end', 'overlap'])

In [27]:
updated_df

,start,end,overlap
0,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTAT...,85
1,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,TCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGT...,70
2,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,AGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAAT...,55
3,AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTAT...,TCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGT...,85
4,AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTAT...,AGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAAT...,70
...,...,...,...
370,AAAAATGGAGATATTATAGTCGACCAAGTATTGGCGTCGAACAACC...,TTGGCGTCGAACAACCGCGCCCTGCAGAATCCCAAGATTCGCCAGG...,70
371,AAAAATGGAGATATTATAGTCGACCAAGTATTGGCGTCGAACAACC...,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,55
372,ATAGTCGACCAAGTATTGGCGTCGAACAACCGCGCCCTGCAGAATC...,TTGGCGTCGAACAACCGCGCCCTGCAGAATCCCAAGATTCGCCAGG...,85
373,ATAGTCGACCAAGTATTGGCGTCGAACAACCGCGCCCTGCAGAATC...,CGCGCCCTGCAGAATCCCAAGATTCGCCAGGCGGCGAACGAGGCCT...,70


Using the updated dictionary, we first find the starting sequence. Then, we look for the next longest match that equals the start (previously appended) string. We append the distinct suffix onto the final string. We repeat this until we arrive at the final string.

In [19]:
final_string = find_start(seq_list,5)[0]
start = find_start(seq_list,5)[0]
y = True

for x in updated_dict: 
    for y in updated_dict: 
        if (y[0] == start): 
            start = y[1]
            #print(start)
            final_string = final_string + (y[1][y[2]:])
    
print(final_string)

CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTCCAAAATAAAGTTTACAGTACCGTCCGGACGGCAGAACTGTCCTCTAGATCGTCCTAACGCCTTAGTCGAATCCCTTGCCGTCGGTAACCACTGAATAAACTACGCGTTAGGACTTTGTCAGACGCGAGGAGCTAGTAGGAGGACAAATCAGCAAACGACCCTGAATTGAACAATGTGAGTAGGTATAACTGTGCTTGTATGACGTCCCGTTCGGTCGTTCTTGAGCAACTTCGGCCAGTGCATGCTATGGGGGAAGCTATGAATTCTATGTTGGAACTTGGGCCCGGCATAGTAGTTTATGCCTGTGGACCGGTGTTGAGTGTATCTGCTGGACCCCGGCGCGTTCACCTGTCCACATCTAATCCAAACATATACTATTGGTATTTGAGCGTCTCACAACGACATCGACTGGTATTAGACACCTACCAGGAACAACCAATCGGTTTAGATGACGCACAGCCACGGACAGCCTCTGTTGCTTGAGCAGTCCCAAAGTGCGTACCTGAAGCCTGCCAAAACGTAGCCTAGGCAAATGCCCGTCGTCTTGCTCATAACTCCTTGGGACTGGCGTATCCATAAATAATCCATTCGATTCCTTGAGAGTTCCACATTAGAGACTTATCCATCGAGGATCAGGCCAAATCCGCGAGACCCGACCGAGATCAAGTATAACTCATTACGCGTGGTGTGGTTGCGGCCCACCCTTATCGTGAGCCAGTTGTTGGATATACCCCTGGGCGGGCCTAAAGCTCCGCAACGAACACCCCCTCCGCTGTGTCTGGTCGATTCTGGCTAGCCGCTCCGTTTGGGTA